# Add Folders to Path

In [1]:
%%time
import sys, os
# get current directory
path = os.getcwd()
# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])
# add Algorithm folder to current working path in order to access the functions inside the folder ´Algorithms´
sys.path.append(parent_directory+"/General_Functions")

CPU times: user 201 µs, sys: 163 µs, total: 364 µs
Wall time: 194 µs


# Double Gyre

## Import Data

In [2]:
%%time
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../Data/double_gyre.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time = mat_file['t']

CPU times: user 339 ms, sys: 89 ms, total: 428 ms
Wall time: 599 ms


## Data/Parameters for Dynamical System

In [3]:
%%time
import numpy as np

# Number of cores to be used for parallel computing
Ncores = 4

# Incompressible/Compressible flow. {True, False}
Incompressible = True

# Periodic boundary conditions
periodic_x = False
periodic_y = False
Periodic = [periodic_x, periodic_y]

## Compute Meshgrid
X, Y = np.meshgrid(x, y)

# List of parameters of the flow.
params_data = {"X": X, "Y": Y, "Time": time, "U": U, "V": V, "Ncores": Ncores, 
               "Incompressible": Incompressible, "Periodic": Periodic}

CPU times: user 766 µs, sys: 1.18 ms, total: 1.95 ms
Wall time: 1.27 ms


<a id='Spatio-Temporal-Domain-of-Dynamical-System'></a>
## Spatio-Temporal Domain of Dynamical System

In [4]:
%%time
# Initial time (in days)
t0 = 0

# Final time (in days)
tN = 10

# time step-size
dt = .1

time = np.arange(t0, tN+dt, dt)

# longitudinal and latitudinal boundaries (in degrees)
xmin = 0
xmax = 2
ymin = 0
ymax = 1

# spacing of meshgrid (in degrees)
dx = 0.05
dy = 0.05

x_domain = np.arange(xmin, xmax + dx, dx)
y_domain = np.arange(ymin, ymax + dy, dy)

X_domain, Y_domain = np.meshgrid(x_domain, y_domain)

params_DS = {"time": time, "X_domain": X_domain, "Y_domain": Y_domain}

CPU times: user 270 µs, sys: 94 µs, total: 364 µs
Wall time: 300 µs


In [5]:
%%time
# Initialize Dynamical System
from ipynb.fs.defs.DynamicalSystem import *
DS = Dynamical_System(params_data, params_DS)

CPU times: user 151 ms, sys: 28 ms, total: 179 ms
Wall time: 268 ms


## Velocity Interpolation

In [6]:
%%time
# Interpolate velocity data using cubic spatial interpolation
DS._Interpolation_velocity("cubic")

CPU times: user 335 ms, sys: 48.4 ms, total: 384 ms
Wall time: 512 ms


## Gradient of Flow map

Trajectories are launched from the grid of initial conditions specified in [Section 2.3](#Spatio-Temporal-Domain-of-Dynamical-System) **(Line 14-17)**.

The temporal domain is specified in [Section 2.3](#Spatio-Temporal-Domain-of-Dynamical-System) **(Line 2-11)**.

In [7]:
%%time
# aux_grid = True --> Use auxiliary grid for numerical computation of gradient.
# Otherwise aux_grid = False.
aux_grid = True

grad_Fmap_grid = DS._grad_Fmap_grid(aux_grid)

  0%|          | 0/21 [00:00<?, ?it/s]

CPU times: user 1.63 s, sys: 667 ms, total: 2.29 s
Wall time: 3min 18s


## Cauchy-Green strain tensor over Meshgrid of Initial Conditions

In [8]:
%%time
B, C = DS._cauchy_green_strain(grad_Fmap_grid)

CPU times: user 15.8 ms, sys: 2.64 ms, total: 18.4 ms
Wall time: 15.8 ms


## Geodesic elliptic LCS

In [9]:
%%time
#from ipynb.fs.defs.geodesic_elliptic_LCS import _geodesic_elliptic_LCS
from ipynb.fs.defs.gradient_C import _gradient_C

lamda = np.arange(0.9, 1.1+0.02, 0.02)

grad_C11, grad_C12, grad_C22 = _gradient_C(X, Y, C)

x0lam, y0lam, phi0lam = _geodesic_elliptic_LCS(X_domain, Y_domain, C, lamda, Incompressible)

NameError: name '_geodesic_elliptic_LCS' is not defined

In [10]:
import matplotlib.pyplot as plt

plt.scatter(x0lam, y0lam)
plt.show()

NameError: name 'x0lam' is not defined